In [5]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urljoin
import time
import random
import pandas as pd

# Base domain for joining relative URLs
BASE_DOMAIN = "https://iowacity.iowaassessors.com/"
# URL for the search page
search_url = "https://iowacity.iowaassessors.com/showResSaleSearch.php?"

# Set up undetected_chromedriver with Chrome version 134
options = uc.ChromeOptions()
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
                     "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36")
driver = uc.Chrome(version_main=134, options=options)
driver.get(search_url)
wait = WebDriverWait(driver, 15)

# --- Fill in the Search Form ---
# Date Range
sale_date_from = wait.until(EC.presence_of_element_located((By.NAME, "sale_date1")))
sale_date_from.clear()
sale_date_from.send_keys("01/01/2023")  # Desired start date
sale_date_to = driver.find_element(By.NAME, "sale_date2")
sale_date_to.clear()
sale_date_to.send_keys("12/30/2023")  # Desired end date

# Occupancy: Single-Family / Owner Occupied (value="1")
occupancy_select = driver.find_element(By.XPATH, "/html/body/form/div/div[6]/div[9]/div[2]/select")
Select(occupancy_select).select_by_value("1")

# Transaction Code: 0 Normal (value="0")
transaction_select = driver.find_element(By.XPATH, "/html/body/form/div/div[6]/div[8]/div[2]/select")
Select(transaction_select).select_by_value("0")

# Class: Residential (value="2")
class_select = driver.find_element(By.XPATH, "/html/body/form/div/div[6]/div[23]/div[2]/select")
Select(class_select).select_by_value("2")

# Click the "Display Results" button
display_results_button = driver.find_element(By.ID, "display_results")
display_results_button.click()

# --- Wait for the Results Page to Load ---
def find_results_table():
    try:
        return wait.until(EC.presence_of_element_located((By.ID, "tblSearchResults")))
    except:
        try:
            return wait.until(EC.presence_of_element_located(
                (By.XPATH, "//table[.//th[contains(text(), 'Sale Date')]]")
            ))
        except Exception as e:
            print("Results table not found using either locator.")
            return None

results_table = find_results_table()
if not results_table:
    print("Results table not found on first page. Exiting.")
    driver.quit()
    exit()

time.sleep(5)  # Extra pause to ensure full load

# --- Scrape the Results and Handle Pagination ---
all_data = []
page_num = 1

# Get the headers first to understand the column positions
header_row = results_table.find_elements(By.XPATH, ".//th")
header_texts = [header.text.strip() for header in header_row]
print("Headers on page:", header_texts)

while True:
    print(f"Scraping page {page_num}...")
    results_table = find_results_table()
    if not results_table:
        print("Results table not found on this page. Exiting loop.")
        break
    
    rows = results_table.find_elements(By.XPATH, ".//tr")
    
    # Debug: print first data row cell values on page 1
    if page_num == 1 and len(rows) > 1:
        first_row_cells = rows[1].find_elements(By.TAG_NAME, "td")
        print("\nDEBUG: First data row cell values:")
        for idx, cell in enumerate(first_row_cells):
            print(f"  Column {idx}: '{cell.text}'")
        print()
    
    # Extract data from rows, starting from index 1 to skip header
    for row in rows[1:]:
        cells = row.find_elements(By.TAG_NAME, "td")
        if cells and len(cells) >= 11:
            # Use the actual table structure based on your provided XPATHs
            # The order from your XPATHs is: Parcel Number, Sale Date, Sale Amount, [Photo], Address, Style, Year Built, Total SF, Lot Area, Appraised Value, Recording
            record = {
                "Parcel Number":    cells[0].text,  # First column
                "Sale Date":        cells[1].text,  # Second column
                "Sale Amount":      cells[2].text,  # Third column
                "Address":          cells[4].text,  # Fifth column (skipping Photo column)
                "Style":            cells[5].text,  # Sixth column
                "Year Built":       cells[6].text,  # Seventh column
                "Total SF":         cells[7].text,  # Eighth column
                "Lot Area":         cells[8].text,  # Ninth column
                "Appraised Value":  cells[9].text,  # Tenth column
                "Recording":        cells[10].text  # Eleventh column
            }
            all_data.append(record)
    
    # Find Next Page link (anchor wrapping an image with alt text "Next Page")
    next_links = driver.find_elements(By.XPATH, "//a[./img[contains(normalize-space(@alt), 'Next Page')]]")
    print(f"Found {len(next_links)} Next Page link(s) on page {page_num}.")
    
    if next_links:
        next_href = next_links[0].get_attribute("href")
        if not next_href:
            print("Next page link found but href is empty. Ending pagination.")
            break
        full_next_url = urljoin(BASE_DOMAIN, next_href)
        print(f"Navigating to next page: {full_next_url}")
        driver.get(full_next_url)
        page_num += 1
        time.sleep(random.uniform(3, 5))
    else:
        print("No 'Next Page' link found. Assuming last page reached.")
        break

# Create DataFrame with columns in the specified order
columns_order = [
    "Parcel Number", "Sale Date", "Sale Amount", "Address", "Style", 
    "Year Built", "Total SF", "Lot Area", "Appraised Value", "Recording"
]

df = pd.DataFrame(all_data)
# Ensure columns are in the correct order
df = df[columns_order]
df.to_csv("historical_transactions_2023_fixed.csv", index=False)

driver.quit()
print("Scraping complete. Sample data:")
print(df.head())

Headers on page: ['Parcel Number', 'Sale Date', 'Sale Amount', 'Photo', 'Address', 'Style', 'Year Built', 'Total SF', 'Lot Area', 'Appraised Value', 'Recording']
Scraping page 1...

DEBUG: First data row cell values:
  Column 0: '1014355009'
  Column 1: '05/25/2023'
  Column 2: '$60,000'
  Column 3: ''
  Column 4: '1521 BROADWAY ST, IOWA CITY'
  Column 5: '1 Story Frame'
  Column 6: '1954'
  Column 7: '1204'
  Column 8: '6,250'
  Column 9: '$150,000'
  Column 10: '6482-625'

Found 1 Next Page link(s) on page 1.
Navigating to next page: https://iowacity.iowaassessors.com/results.php?page=2&history=-2&ts=1743622481
Scraping page 2...
Found 1 Next Page link(s) on page 2.
Navigating to next page: https://iowacity.iowaassessors.com/results.php?page=3&history=-3&ts=1743622481
Scraping page 3...
Found 1 Next Page link(s) on page 3.
Navigating to next page: https://iowacity.iowaassessors.com/results.php?page=4&history=-4&ts=1743622481
Scraping page 4...
Found 1 Next Page link(s) on page 4.
Nav

# Teague Stotlar Project Proposal : Web Scraping

## Web Scraping for 2013, 2018, 2023

In [6]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urljoin
import time
import random
import pandas as pd

BASE_DOMAIN = "https://iowacity.iowaassessors.com/"
search_url = "https://iowacity.iowaassessors.com/showResSaleSearch.php?"

# I am running on Chrome version 134
options = uc.ChromeOptions()
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
                     "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36")

# Scrape data for a given year
def scrape_year_data(year, driver, wait):
    print(f"\n--- Starting scrape for year {year} ---\n")
    
    # Go to search page
    driver.get(search_url)
    time.sleep(3)  
    
    # Date Range
    sale_date_from = wait.until(EC.presence_of_element_located((By.NAME, "sale_date1")))
    sale_date_from.clear()
    sale_date_from.send_keys(f"01/01/{year}") 
    
    sale_date_to = driver.find_element(By.NAME, "sale_date2")
    sale_date_to.clear()
    sale_date_to.send_keys(f"12/30/{year}") 
    
    # Make the occupancy single family / owner occupied (1)
    occupancy_select = driver.find_element(By.XPATH, "/html/body/form/div/div[6]/div[9]/div[2]/select")
    Select(occupancy_select).select_by_value("1")
    
    # Normal values (0)
    transaction_select = driver.find_element(By.XPATH, "/html/body/form/div/div[6]/div[8]/div[2]/select")
    Select(transaction_select).select_by_value("0")
    
    # Residential Class (2)
    class_select = driver.find_element(By.XPATH, "/html/body/form/div/div[6]/div[23]/div[2]/select")
    Select(class_select).select_by_value("2")
    
    # Display Results Button
    display_results_button = driver.find_element(By.ID, "display_results")
    display_results_button.click()
    
    # Wait for Results to load up
    def find_results_table():
        try:
            return wait.until(EC.presence_of_element_located((By.ID, "tblSearchResults")))
        except:
            try:
                return wait.until(EC.presence_of_element_located(
                    (By.XPATH, "//table[.//th[contains(text(), 'Sale Date')]]")
                ))
            except Exception as e:
                print(f"Results table not for year {year}.")
                return None
    
    results_table = find_results_table()
    if not results_table:
        print(f"Results table not found for year {year}.")
        return []
    
    time.sleep(5) 
    
    # Scrape Results
    all_data = []
    page_num = 1
    
    # Get the headers
    header_row = results_table.find_elements(By.XPATH, ".//th")
    header_texts = [header.text.strip() for header in header_row]
    print(f"Headers on page for year {year}:", header_texts)
    
    while True:
        print(f"Scraping page {page_num} for year {year}...")
        results_table = find_results_table()
        if not results_table:
            print(f"Results table not found on page {page_num} for year {year}. Exiting loop.")
            break
        
        rows = results_table.find_elements(By.XPATH, ".//tr")
        
        # Debugging: print first data row cell
        if page_num == 1 and len(rows) > 1:
            first_row_cells = rows[1].find_elements(By.TAG_NAME, "td")
            print(f"\nDEBUG: First data row cell values for year {year}:")
            for idx, cell in enumerate(first_row_cells):
                print(f"  Column {idx}: '{cell.text}'")
            print()
        
        # Extract data from the rows, starting from index 1 to skip header
        for row in rows[1:]:
            cells = row.find_elements(By.TAG_NAME, "td")
            if cells and len(cells) >= 11:
                # Use the actual table structure based on your provided XPATHs
                record = {
                    "Parcel Number":    cells[0].text,  # First column
                    "Sale Date":        cells[1].text,  # Second column
                    "Sale Amount":      cells[2].text,  # Third column
                    "Address":          cells[4].text,  # Fifth column (skipping Photo column)
                    "Style":            cells[5].text,  # Sixth column
                    "Year Built":       cells[6].text,  # Seventh column
                    "Total SF":         cells[7].text,  # Eighth column
                    "Lot Area":         cells[8].text,  # Ninth column
                    "Appraised Value":  cells[9].text,  # Tenth column
                    "Recording":        cells[10].text  # Eleventh column
                }
                all_data.append(record)
        
        # Find Next Page link
        next_links = driver.find_elements(By.XPATH, "//a[./img[contains(normalize-space(@alt), 'Next Page')]]")
        print(f"Found {len(next_links)} Next Page link(s) on page {page_num} for year {year}.")
        
        if next_links:
            next_href = next_links[0].get_attribute("href")
            if not next_href:
                print(f"Next page link found but href is empty for year {year}. Ending pagination.")
                break
            full_next_url = urljoin(BASE_DOMAIN, next_href)
            print(f"Navigating to next page for year {year}: {full_next_url}")
            driver.get(full_next_url)
            page_num += 1
            time.sleep(random.uniform(3, 5))
        else:
            print(f"No 'Next Page' link found for year {year}. Assuming last page reached.")
            break
    
    return all_data

# Main execution
try:
    driver = uc.Chrome(version_main=134, options=options)
    wait = WebDriverWait(driver, 15)
    
    # Years to scrape
    years_to_scrape = [2023, 2018, 2013]
    
    # Column order for CSV
    columns_order = [
        "Parcel Number", "Sale Date", "Sale Amount", "Address", "Style", 
        "Year Built", "Total SF", "Lot Area", "Appraised Value", "Recording"
    ]
    
    # Scrape data each year
    for year in years_to_scrape:
        # Scrape data current year
        year_data = scrape_year_data(year, driver, wait)
        
        # Create DF with scraped data
        df = pd.DataFrame(year_data)
        
        # Make sure columns are in the right order
        df = df[columns_order]
        
        # Save to CSV
        csv_filename = f"historical_transactions_{year}.csv"
        df.to_csv(csv_filename, index=False)
        
        print(f"\nScraped {len(year_data)} records for year {year}.")
        print(f"Data saved to {csv_filename}")
        print(f"Sample data for {year}:")
        print(df.head())
        
        # Add a short delay to make sure code runs smoothly
        if year != years_to_scrape[-1]:
            time.sleep(random.uniform(5, 8))
    
    print("\nAll scraping completed successfully!") # Makes output look cleaner

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    if 'driver' in locals():
        driver.quit()
    print("Driver closed.")


--- Starting scrape for year 2023 ---

Headers on page for year 2023: ['Parcel Number', 'Sale Date', 'Sale Amount', 'Photo', 'Address', 'Style', 'Year Built', 'Total SF', 'Lot Area', 'Appraised Value', 'Recording']
Scraping page 1 for year 2023...

DEBUG: First data row cell values for year 2023:
  Column 0: '1014355009'
  Column 1: '05/25/2023'
  Column 2: '$60,000'
  Column 3: ''
  Column 4: '1521 BROADWAY ST, IOWA CITY'
  Column 5: '1 Story Frame'
  Column 6: '1954'
  Column 7: '1204'
  Column 8: '6,250'
  Column 9: '$150,000'
  Column 10: '6482-625'

Found 1 Next Page link(s) on page 1 for year 2023.
Navigating to next page for year 2023: https://iowacity.iowaassessors.com/results.php?page=2&history=-2&ts=1743623335
Scraping page 2 for year 2023...
Found 1 Next Page link(s) on page 2 for year 2023.
Navigating to next page for year 2023: https://iowacity.iowaassessors.com/results.php?page=3&history=-3&ts=1743623335
Scraping page 3 for year 2023...
Found 1 Next Page link(s) on page 

## Integrating the 3 datasets

In [14]:
df_2013 = pd.read_csv('historical_transactions_2013.csv')
df_2018 = pd.read_csv('historical_transactions_2018.csv')
df_2023 = pd.read_csv('historical_transactions_2023.csv')

# Add a 'Year' column
df_2013['Source Year'] = 2013
df_2018['Source Year'] = 2018
df_2023['Source Year'] = 2023

# Combine
combined_df = pd.concat([df_2013, df_2018, df_2023], ignore_index=True)

# Save
combined_df.to_csv('iowa_city_housing_combined.csv', index=False)